# HMM Implementation in tensorflow

* https://www.ece.ucsb.edu/Faculty/Rabiner/ece259/Reprints/tutorial%20on%20hmm%20and%20applications.pdf

## Train using descent
* DONE

## Predict 
## Visualize model
* TODO

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
import tensorflow_text as text
import numpy as np
from copy import deepcopy

np.set_printoptions(precision=3, suppress=True)

# Forward  Algorithm Tensorflow

* single sequence
* batch of sequences 

In [3]:
from tensorflow.python.framework import ops
from tensorflow.python.data.ops import dataset_ops
from tensorflow.python.util.tf_export import tf_export
from tensorflow.python.data.experimental import group_by_window

from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes

from tensorflow.python.ops import array_ops
from tensorflow.python.ops import check_ops
from tensorflow.python.ops import math_ops

In [ ]:
# Truncate sequences and fit them in buckets
# TODO use ragged tensors instead

In [4]:
def buckettrunc_by_sequence_length(
                              element_length_func,
                              bucket_boundaries,
                              bucket_batch_sizes,
                              drop_remainder=False):   
    
    # Map function
    def element_to_bucket_id(*args):
        """Return int64 id of the length bucket for this element."""
           
        
        bucket_boundaries=[10, 15, 20]
        seq_length = element_length_func(args)
        
        
        err_msg = ("Sequence length (%i) needs to be greater then the first bucket boundary (%i) ."%(seq_length, bucket_boundaries[0]) )
        tf.assert_greater(
            tf.constant(seq_length, dtype=tf.dtypes.int64),
            tf.constant(bucket_boundaries[0], dtype=tf.dtypes.int64),
            message=err_msg)

        
        

        boundaries = sorted(list(bucket_boundaries)) # [10, 15, 20]
        buckets_min = boundaries  # [10, 15, 20]
        buckets_max = boundaries[1:] + [np.iinfo(np.int32).max]  # [15, 20, np.int.max]
        conditions_c = math_ops.logical_and( # for each element, 
            math_ops.greater_equal(x=seq_length, y=buckets_min), # x >= y
            math_ops.less(x=seq_length, y=buckets_max  ))  # x < y
        bucket_id = math_ops.reduce_min(array_ops.where(conditions_c))
        return bucket_id
    
    # Reduce function
    def batching_fn(bucket_id, grouped_dataset):
        batch_size = window_size_fn(bucket_id)  
        boundaries = tf.constant(bucket_boundaries, dtype=tf.dtypes.int64)
        bucket_boundary = boundaries[bucket_id]
        begin = tf.constant(value=0, dtype=tf.dtypes.int64,name='seq_begin')          
        
        grouped_dataset = grouped_dataset.map(lambda seq: tf.slice(seq, begin=[begin], size=[bucket_boundary])) # truncate to bucket boundary
        return grouped_dataset.batch(batch_size, drop_remainder=drop_remainder)
    
    # Batch size functions
    batch_sizes = tf.constant(bucket_batch_sizes, dtype=tf.dtypes.int64)
    def window_size_fn(bucket_id):
        window_size = batch_sizes[bucket_id]
        return window_size
    
    def _apply_fn(dataset):
        return dataset.apply(group_by_window(
            key_func=element_to_bucket_id, 
            reduce_func=batching_fn, 
            window_size_func=window_size_fn)
        )
    
    return _apply_fn

def seq_len(seq):
    return  tf.shape(seq)[0]

In [16]:
# Method playground
# TODO write test

bucket_boundaries=[10, 15, 20]
seq_length = 20

err_msg = ("Sequence length (%i) needs to be greater then the first bucket boundary (%i) ."%(seq_length, bucket_boundaries[0]) )
tf.assert_greater(
    tf.constant(seq_length, dtype=tf.dtypes.int64),
    tf.constant(bucket_boundaries[0], dtype=tf.dtypes.int64),
    message=err_msg)

boundaries = sorted(list(bucket_boundaries)) # [10, 15, 20]
buckets_min = boundaries  # [10, 15, 20]
buckets_max = boundaries[1:] + [np.iinfo(np.int32).max]  # [15, 20, np.int.max]
conditions_c = tf.math.logical_and( # for each element, 
    tf.greater_equal(x=seq_length, y=buckets_min), # x >= y
    tf.math.less(x=seq_length, y=buckets_max  ))  # x < y

bucket_id = tf.math.reduce_min(tf.where(conditions_c))



In [17]:
data = tf.data.TextLineDataset(
    filenames=["data/hmm.dat"], compression_type=None, buffer_size=None, num_parallel_reads=None
)

data = data.map(lambda line: tf.strings.split(line, sep=",") )
data = data.map(lambda line: tf.strings.to_number(line, out_type=tf.dtypes.int32))

data = data.map(lambda line: tf.slice(line, begin=[0], size=[3]  ))
data = data.shuffle(buffer_size=3)

data = data.batch(batch_size=3)

print(list(data.take(1).as_numpy_iterator()))

[array([[0, 2, 0],
       [0, 2, 2],
       [1, 1, 0]], dtype=int32)]


In [18]:
train_sequences = [0] * 2
train_sequences[0] = np.array([
    [0, 2, 0, 1, 2, 2,0,1,1,0,1,2],
    [0, 1, 0, 1, 2, 2,0,0,2,0,1,2],
    [0, 1, 0, 1, 2, 2,0,1,2,0,1,2],
    [0, 2, 2, 0, 2, 1,0,1,0,0,1,2],
    [1, 1, 0, 1, 2, 2,0,1,0,0,1,2],
    [1, 1, 0, 1, 2, 1,0,0,2,0,1,2]
])

train_sequences[1] = np.array([
    [0, 2, 0, 1, 2, 2,0,1,1],
    [0, 1, 0, 1, 2, 2,0,0,2],
    [0, 1, 0, 1, 2, 2,0,1,2],
])

# HMM Model 

* TODO document
* TODO test

In [11]:
class HMM(tf.keras.Model):

    def __init__(self, N=4, M=3, A=None, B=None, PI=None):
        super(HMM, self).__init__()
        
        # TODO create 
        # A state_transistion_probabilities 
        # B emission_probabilities
        # p 
        # Transition probabilities
        if A is None:
            init_A = tf.random.uniform(shape=(N,N), dtype=tf.dtypes.float32)
            init_A = init_A / tf.reduce_sum(init_A, axis=0)
            self.A = tf.Variable(name="state_transition_probabilities", trainable=True, initial_value=init_A)
        else:    
            self.A = tf.Variable(name='state_transition_probabilities', trainable=True, initial_value=tf.constant(
                A, dtype=tf.float32
            ))
                              
        
        # Emission probabilities
        if B is None:
            init_B = tf.random.uniform(shape=(M,N) , dtype=tf.dtypes.float32, seed=None, name=None)
            init_B = init_B / tf.reduce_sum(init_B, axis=0)
            self.B = tf.Variable(name='emission_probabilities', trainable=True, initial_value=init_B)          
        else:   
            self.B = tf.Variable(name='emission_probabilities', trainable=True, initial_value=tf.constant(
               B, dtype=tf.float32
            ))
                                 
        if PI is None:
            # Initial state probabilities
            self.pi = tf.Variable(name='pi', trainable=False, initial_value=tf.constant(
                np.array([1/N]*N), dtype=tf.float32
            ))
        else:
            self.pi = tf.Variable(name='pi', trainable=False, initial_value=tf.constant(
                PI, dtype=tf.float32
            ))
            
        # TODO sanity checks for input data         
        self.normalize()
        
    def call(self, inputs, training=False):
        # inputs are of shape [batch_size, sequence_length]
        
        self.add_loss(self.log_likelihood(inputs))
        return inputs  # this is the negative log-likelihood
    
    
    def log_likelihood(self, obs_batch):
        
        #obs_batch = [batch_size, seq_len]
        batch_size = tf.shape(obs_batch)[0]
        seq_len = tf.shape(obs_batch)[1]     

        def body(i, prev, sum_log_c):
            # prev = [batch_size, N ]
            #tf.print(i, prev, sum_log_c)
   
            o_slice = tf.gather(obs_batch, i, axis=1) # [batch_size]
            #tf.print(o_slice)
            emissions_prob = tf.gather(self.B, o_slice, axis=0) # [batch_size, N]
            #tf.print(emissions_prob)
            batch_A = tf.repeat(tf.expand_dims(tf.transpose(self.A),0), batch_size, axis=0) # [  batch_size, N, N ]
            
            prob_sum = tf.matmul(a=batch_A,b=tf.expand_dims(prev,2))  # [batch_size, N,N] * [batch_size, N, 1] => [ batch_size, N, 1]
            prob_sum = tf.squeeze( prob_sum, 2) #  [ batch_size, N, 1] => [ batch_size, N ]
            
            f = emissions_prob * prob_sum # [ batch_size, N] => [ batch_size, N ]
            
            c = 1. / tf.reduce_sum(f, axis=1) # [ batch_size, N ] => [batch_size]
            c = tf.expand_dims(c, 1) #  [batch_size] => [ batch_size, 1]
            
            next_loopvars =  tf.add(i, 1), f * c, tf.add(sum_log_c, tf.math.log(c))
            
            return next_loopvars

        i = tf.constant(0) # counter for the while loop
        condition = lambda i, prev, sum_log_c: tf.less(i, seq_len ) # abort condition for the while loo p
        
        pis =  tf.repeat(tf.expand_dims(self.pi,0), batch_size, axis=0)
        log_likelihood_sum = tf.zeros(shape=(batch_size,1))
        loopvars = [i,pis,log_likelihood_sum ] 
        
        r = tf.while_loop(condition, body, loopvars)

        return r[2] 
    
    def normalize(self): # to ensure we have probabilities
        self.A.assign(self.A/tf.reduce_sum(self.A, axis=1))
        self.B.assign(self.B/tf.reduce_sum(self.B, axis=0))
        
    def generate(self, steps=1):
        # TODO implement
        pass 
                                             
    def loss(self):
        # forward log loss           
        return lambda y_true, y_predicted : y_predicted # because we predict the negative log likelihood already



# Optimizer

In [12]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
)

hmm = HMM()

# Training 

In [15]:
epochs = 30

steps = 100
# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step in range(steps):
        
        with tf.GradientTape() as tape:
            hmm(train_sequences[step%2])
            loss = tf.reduce_sum(hmm.losses) 
            
        grads = tape.gradient(loss, hmm.trainable_weights)
        optimizer.apply_gradients(zip(grads, hmm.trainable_weights))
        hmm.normalize() # TODO may use callback for that
        
        if (step%10==0):
            print(loss.numpy())
      
 

Start of epoch 0
78.208626
88.41903
97.790955
96.276794
94.4719
93.00278
91.79845
90.78998
89.92912
89.1826
Start of epoch 1
88.52693
87.945206
87.42476
86.95595
86.53122
86.1445
85.79089
85.46636


KeyboardInterrupt: 

# HMM Implementation in Numpy

In [ ]:
class HMM():
    
       
    def __init__(self, N=2, t=0, S=None, M=3, V=None, A=None, B=None, pi=None, train_stop_margin=0.000001):
        #TODO sanity check with N, M and dimension of A, B
        
        self.N = N # number of hidden states in the HMM  (ergodic - each state can be reached from each other state); Number of urns in the example
        self.t = t # the time step we are in
        if S is None:
            self.S = ["S_%i"%s for s in range(self.N)] # an array of states
            self.N = N
        else:
            self.S = S
            self.N = len(S)
        self.q_t = None # the current state at time t
        
        if V is None:
            self.M = M  # number of distinct observation symbols per state, i.e. the discrete alphabet size (colors in the example)
            self.V = ["v_%i"%v for v in range(self.M)] # our alphabet of symbols that can be emitted            
        else:
            self.V = V
            self.M = len(V)
            
        if A is None:
            self.A = np.ones(shape=(self.N,self.N), dtype=np.float64)*(1/self.N) # our state transition probability matrix. an element a_ij contains the probability of transitioning to S_j in the next step, given that in timestep t I am in S_i, how likeli is it 
        else:
            self.A = A
            
        if B is None:
            self.B = np.ones(shape=(self.N,self.M), dtype=np.float64)*(1/self.M) # our symbol emission probability matrix. b_jk contains the probability of emitting symbol v_k when I am in state S_j at timestep t.
        else:
            self.B = B

        if pi is None:
            self.pi = [1/self.N]*self.N # initial state distribution. pi_i contains the probability of starting in state S_i. 
        else:
            self.pi = pi
            
        self.train_stop_margin=train_stop_margin
            
        print(self.N,self.M)
            
    def generate(self, steps=1):
        self.t = 1
        state_trans_prob = self.pi # choose the start state from our starting distribution 
     
        for t in range(1, 1+steps):
            q_t = np.random.choice(self.N,1,p=state_trans_prob)[0]  # transition to next state
            o_t = np.random.choice(self.M,1,p=self.B[q_t,:])[0] # choose the emission probability for the current state
            state_trans_prob = self.A[q_t,:] # get next transition probabilities
            print("Timestep: {}, Current State: {}, Emission: {}".format(t, self.S[q_t], self.V[o_t]))
            
    def visualize(self):
        pass # TODO generate graph and show as image 
    
    def _sequence_valid(self, O):
         # 
        if len(O)==0:
            print("Sequence must have at least one observation, has length 0.")
            return False
        
        #sanity check
        for o in O:
            if o not in self.V:
                print("symbol {} was not in the allowed set of symbol for this HMM: {}".format(o, self.V))
                return False
        
        return True

    """
        Returns P(O|h), i.e., the probability that the sequence of observations O was emitted by this HMM. 
    """
    def sequence_likelihood(self, O=[]):
        if not self._sequence_valid(O): return False
        T = len(O)
        alpha = self.forward(O)
        
        return alpha[T-1,:].sum() # termination step (21)
    
    def forward(self, O=[]):
        if not self._sequence_valid(O): return False
        T = len(O)
        
        alpha=np.ndarray(shape=(T, self.N))
        
        # initialization step (19)
        t=0
        for i in range(self.N):
            alpha[t,i] = self.pi[i]*self.B[i,self.V.index(o)]
        
        for t, in range(0, T-1): # induction step (20)
            o = O[t+1]
            for j in range(self.N):
                state_trans_prob = sum([ alpha[t,i]*self.A[i,j] for i in range(self.N) ])       
                alpha[t+1,j] =  state_trans_prob * self.B[j,self.V.index(o)]              
                
        return alpha
        
    
    def backward(self, O=[]):
        if not self._sequence_valid(O): return False
        T = len(O)
        beta = np.ndarray(shape(T, self.N))
        
        beta[T-1,:] = 1
         
        for t in range(T-2, -1, -1):             
            for i in range(self.N):
                beta[t,i] = sum( self.A[i,j]*self.B[j,self.V.index(O[t+1])]*beta[t+1,j] for j in range(self.N)  )

        return beta
    
    def xi(self, O[]): # (36) the probability of being in state S_i at time t, and S_j at time t+1
        if not self._sequence_valid(O): return False
        T = len(O)
        alpha = self.forward(O)
        beta =  self.backward(O)
        
        xi = np.ndarray(T-1, self.N, self.N)
        
        for t, in range(0, T-1): # (37)
            for i in range(self.N):
                for j in range(self.N):
                    xi[t, i, j] =  alpha[t, i] * self.A[i,j] *  self.B[j,self.V.index(O[t+1])] * beta[t+1, j]
            
            xi[t,:,:]=xi[t,:,:]/xi[t,:,:].sum()  
            
    def gamma(self, xi): # estimated  probability of being in a state
        T_1 = xi.shape[0] # (38)               
        gamma = np.ndarray(shape=(T_1, self.N))
        for t in range(T_1):
            for i in range(self.N):
                gamma[t,i] = xi[t, i,:].sum()     
    
    
    """
        if you want to train multiple sequences, you need to add summation over R to all the terms 
    """
            
    def train(self, O[]):
        
        if not self._sequence_valid(O): return False
        # xi_ij = the probability of being in state S_i at the time t, and S_j at the time t+1(
        T = len(O)
        
        loop=True
        while loop:       
            # 1) create new model h' = (A',B',pi') as a copy of h (A,B, pi)        
            h_bar = deepcopy(self)

            # 2) update parameters of h' 
            xi = h_bar.xi(O)
            gamma = self.gamma(xi)

            h_bar.pi = gamma[0,:].tolist() # (40a)
            h_bar.A = xi.sum(axis=0) / gamma.sum(axis=0) # (40b) sum over t ()


            for k in range(self.M): # (40c)
                for j in range(self.N):
                    h_bar.B[k,j] = gamma[np.where(np.array(O)==self.V[k]),j] / gamma.sum(axis=0, j) # sum over t       

            # 3.1) if  h is similar enough h', abort 

            P_h_given_O = self.sequence_likelihood(O)
            P_h_bar_given_O = h_bar.sequence_likelihood(O)
            
            updated_model_is_better=P_h_bar_given_O>P_h_given_O and P_h_bar_given_O-P_h_given_O>self.train_stop_margin

            if (updated_model_is_better):
                self.update_from(h_bar)
            else:
                return True
           
    def update_from(self, hmm):
        self.A = hmm.A
        self.B = hmm.B
        self.pi = hmm.pi      
    
    def best_state_sequence(self, O=[]):
        if not self._sequence_valid(O): return False
        
        for t, o in enumerate(O):
            if t==0: # Initialization
                delta = [self.pi[i]*self.B[i,self.V.index(o)] for i in range(self.N)] # (32a)                
                psi = np.ndarray(shape=(len(O),self.N), dtype=np.int) 
                psi[t, :]=0 # (32b)
            else: # Recursion
                new_delta = [0]*self.N
                for j in range(self.N):
                    new_delta[j] = max( [delta[i]*self.A[i,j]  for i in range(self.N)] ) * self.B[j,self.V.index(o)] # (33a)
                    psi[t, j] = np.argmax( [delta[i]*self.A[i,j]  for i in range(self.N)] ) # (33b)
                delta = new_delta
            print(delta)
    
        P_star = max(delta) # (34a)
        q_star = np.argmax(delta) # (34b)
        
        path = [-1]*len(O)
        path[-1]=q_star  # last element of our sequence is q_star
        next_q = q_star
        print(path)
  
        
        for t in range( len(O)-2, -1, -1): # we use -1 becase we already have q_star in our list. all the other values need to be an extra -1 because of how python reverse counting works.
            print(t)
            next_q = psi[t+1, next_q] # (35)
            path[t] = next_q
                                    
        return [self.S[p] for p in path]                     
                     
        
        
        
        
        
        
        

## Generating a sequence of observations

In [ ]:
h = HMM()
h.generate(5)


In [ ]:
V = ["v_0", "v_1", "v_2", "v_3", "v_4"]
O = ["v_2","v_1", "v_4", "v_1"]

k = 1

mask = np.zeros(len(O))
mask[np.where(np.array(O)==V[k])]=1
mask

In [ ]:
np.array([]).tolist()

In [ ]:
np.argmax([1,1,1])

# Given a sequence of observation, how likely is it that our model emitted such a sequence
* Problem 1 in the paper - calculate P(O|$\lambda$), where $\lambda$= (A, B, pi)
* Allows us to choose the model which best matches an observation

In [ ]:
h = HMM()

h.sequence_likelihood(O)

In [ ]:
# Sanity Check -sum of all combinations has to be 1

In [ ]:
from itertools import product
chain_length = 3  # any int > 0
all_combos = list(product(*(h.V,) * len(O)))
all_likelihoods = list(map(lambda o: h.sequence_likelihood(o), all_combos))
print("All possible all_likelihoods added: {}.".format(sum(all_likelihoods)))

# Give a model and an observation seqeunce, what is the optimal state transition sequence 
* problem 2 in paper
* given $O$, calculate the most likely $Q$
* multiple correct answers
* Viterbi Algorithm

In [ ]:
A=np.array([
    [0.05,0.05,0.9],# from S0, goto S2
    [0.1,0.7,0.2],# from S1, goto S1
    [0.6,0.1,0.3] # from S2, goto S0
])
A

In [ ]:
B=np.array([
    [0.05,0.05, 0.8, 0.05, 0.05], # from S0, emit v2
    [0.1 ,0.5 , 0.2, 0.1 , 0.1 ], # from S1, emit v1
    [0.1 ,0.1 , 0.1, 0.1 , 0.6 ]  # from S2, emit v4
])
B

In [ ]:
h = HMM(A=A, B=B, M=5, N=3)
  
        
O = ["v_2","v_1", "v_4", "v_1"]
h.best_state_sequence(O)

# Learning the Parameters of the HMM using baum welch algorithm
* EM expectation modification method
* iterative approach, stops at maximum likelihood. leads to local maxima only